# **Miniproject 2**
## **~Large~ Small Language Model**

### **Objective**
Implement a transformer-based, character-level language model (GPT-like) and train it on the Shakespeare dataset. By the end of this project, you should be able to generate Shakespearean-like text given a seed string.

You will probably want to train the model on a GPU. You can use free GPUs on [Google Colab](https://colab.research.google.com/?utm_source=scs-index).

### **Dataset**:

The Shakespeare dataset contains the complete works of William Shakespeare, including his plays, poems, and sonnets.

[**Download link**](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)

In a character-level language model, each character in the input data is mapped to its respective index from a dictionary. The input to the model is in the form (B, N), where B is the batch size and N is the number of tokens for each sequence. The model was tested with B=N=128, but feel free to explore different values.

An interface for the dataset class that takes care of tokenization is provided below.



```python
from torch.utils.data import Dataset

class CharDataset(Dataset):
    """
    Emits batches of characters.

    Adapted from "https://github.com/karpathy/minGPT".
    """

    def __init__(self, config, data):

        chars = ... # get characters from the input data
        self.stoi = { ch:i for i,ch in enumerate(chars) } # map characters to integer indices

        ...

    def get_vocab_size(self):
        raise NotImplementedError()

    def __len__(self):
        raise NotImplementedError()

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        # encode every character to an integer
        # return the chunk and the shifted version as tensors
        pass
```




### **Requirements**

#### **Architecture**

Implement the Transformer's decoder-only structure.
This includes

* input token embeddings
* the causal multi-head self-attention mechanism
* feed-forward neural networks
* positional encodings, residual connections, layer normalizations.

The project was tested with $12$ layers, $8$ attention heads, and $768$ embedding dimensions, on a single GPU.

The `forward` method for the entire model has the following form:

```
tok_emb = WTE(idx) # token embeddings
pos_emb = WPE(pos) # position embeddings
x = Dropout(tok_emb + pos_emb)
for Block in Blocks:
    x = Block(x)
x = Final_LayerNorm(x)
logits = LM_Head(x)
```

The `forward` method for the transformer block has the following form:



```
x = x + self.CausalSelfAttn(self.LayerNorm_1(x))
out = x + self.MLP(self.LayerNorm_2(x))
```

---

#### **Training**

In a character-level transformer language model, the goal is to predict the next character in a sequence given the previous characters. To train such a model effectively, we use two versions of our data: the input sequence and a shifted version of this sequence, which serves as the target for our predictions.

Preprocess the dataset to a character-level representation.
Use a sliding window approach for sequence chunks (e.g., window size of $128$ characters).
Implement causal masking for the self-attention mechanism.
Use the [Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) optimizer and the cross-entropy loss.

**Optional**:

* Implement a learning rate decay strategy
* Implement gradient clipping

---


#### **Evaluation and Inference**

* Monitor the cross-entropy loss. Use a seed string to initialize the model and generate Shakespearean-like text.

* In order to generate the characters, at each generation step you can either select the character with the highest probability, or you can sample according to the output distribution.

The high-level pseudocode for generation is:

```python
model.eval()
with torch.no_grad():
    context = "O God, O God!"
    tokenized_context = tokenize(context)
    # the model should implement a method to generate tokens given a prompt
    y = model.generate(tokenized, ...)
    completion = tokens_to_string(y)
```

**Optional**:
* Compute the [perplexity](https://medium.com/@priyankads/perplexity-of-language-models-41160427ed72#:~:text=Intuitively%2C%20perplexity%20means%20to%20be,loss%20obtained%20from%20the%20model.) metric for quantitative evaluation.

### **Example Outputs**

The following are my outputs after $6000$ steps of training, with the seed string "O God, O God!"



```
O God, O God! neither? unto the base very ears,
As damned with it.

DUKE OF YORK:
Away! Once more, one word.

RICHARD:
Clove, dear so; and therein my son will be
false of woe: if ye seems to be the mother
Of gracious order this time when R going kinsperse eyes,
What dost bewreck her fairer drying tears.

NORTHUMBERLAND:
Have you forgot the Duke of Norfolk, get him to
again; and and agilic: there is my spirit
So maly did must such a marble perfection.

ELBOW:
Come, bring them with oaths, and so deliver
```


### Resources:

* Vaswani et al., "Attention is All You Need": [link](https://arxiv.org/abs/1706.03762)

* Illustrated Transformer by Jay Alammar: [link](https://jalammar.github.io/illustrated-transformer/)

* OpenAI GPT-2 Paper: [link](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

* Deep Learning Course slides on transformers: [link](https://fleuret.org/dlc/materials/dlc-handout-13-3-transformers.pdf)

In [ ]:
import torch
import time
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
device = torch.device("cuda")

In [ ]:
print(torch.__version__)
if torch.cuda.is_available():
  print("GPU is available")
  print("CUDA version:", torch.version.cuda)
else:
  print("GPU is not available")

2.5.1+cu121
GPU is available
CUDA version: 12.1


In [ ]:
class CharDataset(Dataset):
  def __init__(self, config, data):
    # create dictionary of chars
    chars = sorted(list(set(data)))
    self.stoi = {ch: i for i, ch in enumerate(chars)}
    self.itos = {i: ch for i, ch in enumerate(chars)}
    self.vocab_size = len(chars)
    # get data from the text
    self.data = [self.stoi[ch] for ch in data]
    self.block_size = config['block_size']

  def get_vocab_size(self):
    return self.vocab_size

  def __len__(self):
    return len(self.data) - self.block_size

  def __getitem__(self, idx):
    # grab a chunk of (block_size + 1) characters from the data
    chunk = self.data[idx:idx + self.block_size + 1]
    # encode every character to an integer(long)
    # return the chunk and the shifted version as tensors
    x = torch.tensor(chunk[:-1], dtype=torch.long)  # current sequence
    y = torch.tensor(chunk[1:], dtype=torch.long)  # target sequence
    return x, y


In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self):
    super(MultiHeadAttention, self).__init__()
    self.W_Q = nn.Linear(d_model, d_k * n_heads, bias = False)
    self.W_K = nn.Linear(d_model, d_k * n_heads, bias = False)
    self.W_V = nn.Linear(d_model, d_v * n_heads, bias = False)
    self.fc = nn.Linear(n_heads * d_v, d_model, bias = False)
    self.layer_norm = nn.LayerNorm(d_model)

  def forward(self, input_Q, input_K, input_V, attn_mask):
    residual, batch_size = input_Q, input_Q.size(0)
    Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
    K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
    V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)
    attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
    context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
    context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v)
    output = self.fc(context)
    return self.layer_norm(output + residual), attn

class PoswiseFeedForwardNet(nn.Module):
  def __init__(self):
    super(PoswiseFeedForwardNet, self).__init__()
    self.fc = nn.Sequential(
      nn.Linear(d_model, d_ff, bias=False),
      nn.ReLU(),
      nn.Linear(d_ff, d_model, bias=False)
    )
    self.layer_norm = nn.LayerNorm(d_model)

  def forward(self, inputs):
    residual = inputs # inputs : [batch_size, len_q, d_model]
    output = self.fc(inputs)
    return self.layer_norm(output + residual)

def get_attn_pad_mask(seq_q, seq_k):
  batch_size, len_q = seq_q.size()
  batch_size, len_k = seq_k.size()
  pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k, one is masking
  return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

def get_attn_subsequent_mask(seq):
  """
  seq: [batch_size, tgt_len]
  """
  attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
  # attn_shape: [batch_size, tgt_len, tgt_len]
  subsequence_mask = np.triu(np.ones(attn_shape), k=1)
  subsequence_mask = torch.from_numpy(subsequence_mask).byte()
  subsequence_mask = subsequence_mask.to(device)
  return subsequence_mask  # [batch_size, tgt_len, tgt_len]


class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super(ScaledDotProductAttention, self).__init__()

  def forward(self, Q, K, V, attn_mask):
    scores = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(Q.size(-1))
    scores.masked_fill_(attn_mask, -1e9)
    attn = nn.Softmax(dim = -1)(scores)
    context = torch.matmul(attn, V)
    return context, attn


class PositionalEncoding(nn.Module):
  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)

    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    # pe:[max_len*d_model]

    pe = pe.unsqueeze(0).transpose(0, 1)
    # pe：[max_len*1*d_model]
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + self.pe[:x.size(0), :] # x: [seq_len, batch_size, d_model]
    return self.dropout(x)

class DecoderLayer(nn.Module):
  def __init__(self):
    super(DecoderLayer, self).__init__()
    self.dec_self_attn = MultiHeadAttention()
    self.pos_ffn = PoswiseFeedForwardNet()

  def forward(self, dec_inputs, dec_self_attn_mask):
    dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, attn_mask=dec_self_attn_mask)
    dec_outputs = self.pos_ffn(dec_outputs)
    return dec_outputs, dec_self_attn

class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.tgt_emb = nn.Embedding(vocab_size, d_model)
    self.pos_emb = PositionalEncoding(d_model)
    self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])
    self.final_layer_norm = nn.LayerNorm(d_model)

  def forward(self, dec_inputs): # dec_inputs : [batch_size x target_len]
    dec_outputs = self.tgt_emb(dec_inputs)  #dec_outputs  [batch_size, tgt_len, d_model]
    dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, tgt_len, d_model]

    dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)
    dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs)

    ## When two matrices are added together, those greater than 0 are 1, those less than 0 are 0, and those with 1 are then filled to infinity
    dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

    dec_self_attns = []
    for layer in self.layers:
      dec_outputs, dec_self_attn = layer(dec_outputs, dec_self_attn_mask)
      dec_self_attns.append(dec_self_attn)
    dec_outputs = self.final_layer_norm(dec_outputs)
    return dec_outputs, dec_self_attns

class GPT(nn.Module):
  def __init__(self):
    super(GPT, self).__init__()
    self.decoder = Decoder()
    self.projection = nn.Linear(d_model, vocab_size, bias=False)
    # Initialize weights
    self.apply(self._init_weights)
  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      nn.init.normal_(module.weight, mean=0, std=0.02)
      if module.bias is not None:
        nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      nn.init.normal_(module.weight, mean=0, std=0.02)
  def forward(self, dec_inputs):
    dec_outputs, dec_self_attns = self.decoder(dec_inputs) # [batch_size, tgt_len, d_model]
    dec_logits = self.projection(dec_outputs)  # [batch_size, tgt_len, vocab_size]
    return dec_logits.view(-1, dec_logits.size(-1)), dec_self_attns

In [ ]:
def check_labels(dec_outputs):
  if (dec_outputs < 0).any() or (dec_outputs >= vocab_size).any():
    print(f"Invalid target label values detected! Min: {dec_outputs.min()}, Max: {dec_outputs.max()}")
    # Print part of the label data for easy debugging
    print(f"Some labeled data: {dec_outputs[:10]}")
    raise ValueError(f"Tag out of bounds!The label maximum is {dec_outputs.max()}, and the vocab_size is {vocab_size}.")

def train_step(model,data_loader,optimizer,criterion,clip=1,print_every=None):
  model.train()
  if print_every == 0:
    print_every = 1
  print_loss_total = 0
  epoch_loss = 0
  for i, (dec_inputs, dec_outputs) in enumerate(tqdm(data_loader)):
    optimizer.zero_grad()
    dec_inputs, dec_outputs =dec_inputs.to(device), dec_outputs.to(device)
    print(f"dec_inputs shape: {dec_inputs.shape}")
    print(f"dec_outputs shape: {dec_outputs.shape}")
    # Check whether the target label is out of bounds
    check_labels(dec_outputs)
    # outputs: [batch_size * tgt_len, tgt_vocab_size]
    outputs, dec_self_attns = model(dec_inputs)
    print(f"model output shape: {outputs.shape}")
    loss = criterion(outputs, dec_outputs.view(-1))
    print_loss_total += loss.item()
    epoch_loss += loss.item()
    loss.backward()
    # gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    if print_every and (i + 1) % print_every == 0:
      print_loss_avg = print_loss_total / print_every
      print_loss_total = 0
      print('\tCurrent Loss: %.4f' % print_loss_avg)
  return epoch_loss / len(data_loader)

def compute_validation_loss(model, data_loader, criterion):
  model.eval()
  val_loss = 0
  total_samples = 0
  with torch.no_grad():
    for dec_inputs, dec_outputs in tqdm(data_loader, desc="Validating"):
      dec_inputs, dec_outputs = dec_inputs.to(device), dec_outputs.to(device)
      check_labels(dec_outputs)
      # Forward pass
      outputs, _ = model(dec_inputs)
      # Compute loss
      loss = criterion(outputs.view(-1, outputs.size(-1)), dec_outputs.view(-1))  #Flattened into one dimension
      val_loss += loss.item() * dec_inputs.size(0)  # Cumulative total loss, multiplied by batch size
      total_samples += dec_inputs.size(0)  # Total number of samples
  return val_loss / total_samples

def train(model, train_loader, val_loader, epochs):
  criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  for epoch in range(epochs):
    start_time = time.time()
    train_loss = train_step(model, train_loader, optimizer, criterion, CLIP, print_every=10)
    val_loss = compute_validation_loss(model, val_loader, criterion)
    end_time = time.time()
    checkpoint_path = f'checkpoints_epoch_{epoch}.pt'
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {end_time - start_time:.2f}s")

In [ ]:
def load_data(file_path, config, batch_size=256):
  with open(file_path, 'r') as f:
    text_data = f.read()
  # Split data into train (60%), validation (20%), and test (20%)
  train_split = int(len(text_data) * 0.6)
  val_split = int(len(text_data) * 0.8)
  train_data = text_data[:train_split]
  val_data = text_data[train_split:val_split]
  test_data = text_data[val_split:]
  train_dataset = CharDataset(config, train_data)
  stoi = train_dataset.stoi
  vocab_size = train_dataset.get_vocab_size()
  val_dataset = CharDataset(config, val_data)
  test_dataset = CharDataset(config, test_data)
  # Create DataLoaders
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  print(f"Vocabulary size: {train_dataset.get_vocab_size()}")
  print(f"Train data length: {len(train_data)}")
  print(f"Validation data length: {len(val_data)}")
  return train_dataloader, val_dataloader, test_dataloader

In [ ]:
if __name__ == '__main__':
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # Configurations
  config = {'block_size': 128}
  # Load data
  train_dataloader, val_dataloader, test_dataloader = load_data('input.txt', config)
  # Model parameters
  d_model = 768  # Embedding Size
  d_ff = 2048  # FeedForward dimension
  n_layers = 12  # Number of Decoder Layers
  n_heads = 8  # Number of heads in Multi-Head Attention
  CLIP = 1
  epochs = 5
  vocab_size = train_dataloader.dataset.vocab_size
  # Initialize model
  model = GPT().to(device)
  # Train model (includes validation)
  train(model, train_dataloader, val_dataloader, epochs)

Vocabulary size: 64
Train data length: 669236
Validation data length: 223079


  0%|          | 0/2614 [00:00<?, ?it/s]

dec_inputs shape: torch.Size([256, 128])
dec_outputs shape: torch.Size([256, 128])
model output shape: torch.Size([32768, 64])


  0%|          | 1/2614 [00:00<16:26,  2.65it/s]

dec_inputs shape: torch.Size([256, 128])
dec_outputs shape: torch.Size([256, 128])
model output shape: torch.Size([32768, 64])


...

100%|█████████▉| 2613/2614 [32:44<00:00,  1.33it/s]

dec_inputs shape: torch.Size([180, 128])
dec_outputs shape: torch.Size([180, 128])
model output shape: torch.Size([23040, 64])


Validating:   6%|▋         | 55/871 [00:15<03:49,  3.56it/s]

Invalid target label values detected! Min: 0, Max: 64
Some labeled data: tensor([[52, 42,  6,  ..., 42,  1, 58],
        [42,  6,  1,  ...,  1, 58, 46],
        [ 6,  1,  4,  ..., 58, 46, 43],
        ...,
        [ 0,  0, 25,  ..., 53, 59, 56],
        [ 0, 25, 27,  ..., 59, 56, 58],
        [25, 27, 26,  ..., 56, 58, 46]], device='cuda:0')


ValueError: Tag out of bounds!The label maximum is 64, and the vocab_size is 64.

In [ ]:
def generate_text(model, start_text, length, dataset):
  model.eval()
  generated = start_text
  input_ids = torch.tensor([dataset.stoi[ch] for ch in start_text]).unsqueeze(0).to(device)
  for _ in range(length):
    with torch.no_grad():
      logits, _ = model(input_ids)
      predicted_id = torch.argmax(logits[0, -1]).item()  # Get the predicted next token

    generated += dataset.itos[predicted_id]  # Convert the index to a character
    input_ids = torch.cat([input_ids, torch.tensor([[predicted_id]]).to(device)], dim=1)  # Append the predicted token to input_ids
  return generated

start_text = "O God, O God!"
dataset = test_dataloader.dataset
generate_text(model,start_text, length = 100, dataset=dataset)
print(generated_text)

O God, O God!






































































































In [ ]:
def evaluate(model, data_loader, criterion):
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for dec_inputs, dec_outputs in data_loader:
      dec_inputs, dec_outputs = dec_inputs.to(device), dec_outputs.to(device)
      outputs, _ = model(dec_inputs)
      loss = criterion(outputs, dec_outputs.view(-1))
      total_loss += loss.item()
  return total_loss / len(data_loader)

test_loss = evaluate(model, test_dataloader, nn.CrossEntropyLoss(ignore_index=0).to(device))
print("The loss of test data set is:", test_loss)

The loss of test data set is: 5.535172477792242
